# Creació dataset 
###  Dataset complert per a la implementació de tècniques d'aprenentatge profund

In [1]:
"""
Dependencies:
numpy
pandas
"""
import os
import numpy as np
import pandas as pd

In [2]:
# recollim tots els fitxers en un únic llistat
escenaris = [file for file in os.scandir("Scenarios")]

In [3]:
# inicialització del dataset final
final_dataset = pd.DataFrame(columns = ["Node", "NumCont", "DURC", "ICT", "Escenari"])

In [4]:
# llibreria de funcions

def find_mean(temps):
    """Calculates the mean pair value given a list"""
    # If there is no info in the list, then there is no full circle of connections
    # and so provisionally, we count as 0 (because we cannot divide by 0 later)
    if len(temps) == 0:
        return 0
    
    temps = temps.tolist()
    
    # Find each pair, compute the time between itemps and return its mean value
    t_connected = 0
    for i in range(0, len(temps), 2):
        inici = temps[i]
        final = temps[i+1]
        t_connected += (final - inici)
        
    return t_connected/(len(temps)/2)


def mean_btw_conns(temps):
    """To calculate the mean time between different connections (ICT)"""
    # If the list is even then the last value is a "down" item and we take it
    # down to not disturbe the calculations
    if len(temps)%2 == 0:
        temps = temps[:-1]
    
    # We take down the first item in the list so we have inversed pairs to calculate
    # the time mean
    temps = temps[1:]
    
    return find_mean(temps)

def mean_conn_duration(temps):
    """To calculate the mean time of the connections's duration (DURC)"""
    # If the list is not even then the last communication did not conclude
    # provisionally, we take it down
    if len(temps)%2 != 0:
        temps = temps[:-1]
    
    return find_mean(temps)

In [14]:
# itera per tots els fitxers i extreu les dades
for xarxa, escenari in zip(escenaris, range(1, len(escenaris) + 1)):
                           
    dataset = pd.read_csv(xarxa, sep=" ", header=None)
    dataset.columns = ["Time", "CONN", "Node", "Node2", "Type"]
    dataset.drop(["CONN"], axis = 1, inplace = True)
    
    one_file = dataset.groupby(['Node'])["Node2"].size().reset_index(name ='NumCont')
    
    temps = dataset.groupby(['Node', 'Node2']).agg(pair_DURC = ("Time", mean_conn_duration), pair_ICT = ("Time", mean_btw_conns)).reset_index()    
    final_attrs = temps.groupby(['Node']).agg(DURC = ("pair_DURC", np.mean), ICT = ("pair_ICT", np.mean)).reset_index()

    one_file = one_file.merge(final_attrs, how = 'inner')
    one_file["Escenari"] = escenari
    
    final_dataset = pd.concat([one_file, final_dataset], ignore_index=True)

    print("File", escenari, "done!")

File 1 done!
File 2 done!
File 3 done!
File 4 done!
File 5 done!
File 6 done!
File 7 done!
File 8 done!
File 9 done!
File 10 done!
File 11 done!
File 12 done!
File 13 done!
File 14 done!
File 15 done!
File 16 done!
File 17 done!
File 18 done!
File 19 done!
File 20 done!
File 21 done!
File 22 done!
File 23 done!
File 24 done!
File 25 done!
File 26 done!
File 27 done!
File 28 done!
File 29 done!
File 30 done!
File 31 done!
File 32 done!
File 33 done!
File 34 done!
File 35 done!
File 36 done!
File 37 done!
File 38 done!
File 39 done!
File 40 done!
File 41 done!


In [19]:
# comproba que tots els camps apareguin bé
final_dataset.head()

,Node,NumCont,DURC,ICT,Escenari
0,0,12672,27.851217,2256.327125,41
1,1,11940,27.204866,2226.972516,41
2,2,11780,26.653695,2409.885983,41
3,3,12844,20.685742,2302.824219,41
4,4,13218,23.920808,2265.509583,41


In [23]:
# guarda el dataset complert en un fitxer a part
final_dataset.to_csv("all_scenarios.csv", encoding='utf-8', index=False)